In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import os

parent_path = str(Path("../").resolve())
print(parent_path)
sys.path.append(parent_path)

from seads_envs import load_env

In [ ]:
os.system("rm -r board_game_gif")
os.makedirs("board_game_gif")

def render_sequence(env, env_name, push_sequence, get_push_loc_fcn):
    env.unwrapped._discrete_env._zero_board()
    for action in reversed(push_sequence):
        env.unwrapped._discrete_env.step(action)
    
    for idx, action in enumerate(push_sequence):
        # plot board before action
        fig, ax = plt.subplots(nrows=1, ncols=1)
        env.plot_board(ax, board_state=env.board_state)
        ax.scatter(*get_push_loc_fcn(action), marker="x", s=250, linewidth=5, color="darkblue")
        plt.savefig(f"board_game_gif/{env_name}_{idx*2:03}.png", bbox_inches="tight")

        env.unwrapped._discrete_env.step(action)

        # plot board after action
        fig, ax = plt.subplots(nrows=1, ncols=1)
        env.plot_board(ax, board_state=env.board_state)
        ax.scatter(*get_push_loc_fcn(action), marker="x", s=250, linewidth=5, color="darkblue")
        plt.savefig(f"board_game_gif/{env_name}_{idx*2+1:03}.png", bbox_inches="tight")
    
    os.system(
        "ffmpeg -hide_banner -loglevel error -pattern_type glob "
        f"-i 'board_game_gif/{env_name}_*.png' -vf palettegen board_game_gif/palette_{env_name}.png"
    )
    os.system(
        "ffmpeg -hide_banner -loglevel error -framerate 1 "
        "-loop 0 -pattern_type "
        f"glob -i 'board_game_gif/{env_name}_*.png' "
        f"-i board_game_gif/palette_{env_name}.png "
        f"-lavfi paletteuse board_game_gif/{env_name}.gif"
    )


In [ ]:
# LightsOut
env_name = "LightsOutCursorEnvBs5"
reset_split = "train"
env_kwargs = {
    "max_solution_depth": 1, 
    "random_solution_depth": False, 
    "max_steps": 10, 
}
env = load_env(env_name, reset_split, env_kwargs)

push_sequence = [12, 18, 17, 9, 22]

def get_push_loc_lightsout(action):
    row = action // 5
    col = action - row * 5
    push_loc = 0.5+(col-2)/5, 0.5+(row-2)/5
    return push_loc

render_sequence(env, "lightsout", push_sequence, get_push_loc_lightsout)

<img src="board_game_gif/lightsout.gif" width="250" align="center">

In [ ]:
# TileSwap
env_name = "TileSwapCursorEnvBs3"
reset_split = "train"
env_kwargs = {
    "max_solution_depth": 1, 
    "random_solution_depth": False, 
    "max_steps": 10, 
}
env = load_env(env_name, reset_split, env_kwargs)

push_sequence = [1, 4, 2, 5, 7]

pivot_points = np.array(env.unwrapped.board._pivot_points)
swap_indices = env.unwrapped.board._swap_indices

def get_push_loc_tileswap(action):
    return pivot_points[action]

def step_tileswap(action):
    env.unwrapped._discrete_env.swap(*swap_indices[action])

env.unwrapped._discrete_env.step = step_tileswap
    
render_sequence(env, "tileswap", push_sequence, get_push_loc_tileswap)

<img src="board_game_gif/tileswap.gif" width="250" align="center">